### **Ejercicio N°01: RUSLE a Nivel Mundial**
  <img src="https://user-images.githubusercontent.com/16768318/73690808-1604b700-46c9-11ea-8bdd-43e0e490a0a3.gif" align="right" width = 60%/>

Genere una función para calcular la Ecuacion Universal de Perdida de Suelo (RUSLE) para cualquier parte del mundo. La funcion debe tener los siguientes parametros. **rusle()**



In [0]:
#@title Credenciales Google Earth Engine
import os 
credential = '{"refresh_token":"1//05P739font3A1CgYIARAAGAUSNwF-L9IrQLJR51eiG045wEQTaEY58dcZqNDMR9DX_Kkh_guasDrd8dvft2a7AXhfq1hBk20FCLA"}'
credential_file_path = os.path.expanduser("~/.config/earthengine/")
os.makedirs(credential_file_path,exist_ok=True)
with open(credential_file_path + 'credentials', 'w') as file:
    file.write(credential)

In [0]:
import ee
ee.Initialize()

In [0]:
#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

### **1) Factor R**

El **factor R** es el factor de erosividad de la lluvia. Este factor indica el potencial erosivo de la lluvia que afecta en el proceso de erosión del suelo. Haciendo una analogía, se podría decir que una lluvia fuerte un día al año puede producir suficiente energía para erosionar el suelo que varias lluvias de mediana intensidad a lo largo de un año.

El factor erosividad (R) es definido como la sumatoria anual de los promedios de los valores individuales del índice de tormenta de erosión (EI30). Donde E es la energía cinética por unidad de área e I30 es la máxima intensidad en 30 minutos de precipitación. Esto se puede definir en la siguiente ecuación: 

<img src="https://user-images.githubusercontent.com/16768318/73694650-67fd0b00-46d0-11ea-87f6-4ed9501cf964.png"  width = 60%>

Por tanto, la energía de la tormenta (EI o R) indica el volumen de lluvia y escurrimiento, pero una larga y suave lluvia puede tener el mismo valor de E que una lluvia de corta y más alta intensidad. (Mannaerts, 1999). La energía se calcula a partir de la fórmula de Brown y Foster: 

<img src="https://user-images.githubusercontent.com/16768318/73694782-b3171e00-46d0-11ea-94fe-94f3f57941c5.png"  width = 40%>

A partir de la ecuación anterior, el cálculo del factor R es un proceso complejo y requiere datos horarios o diarios de varios años. Por lo que se han desarrollado diferentes ecuaciones que adaptan la erosividad local mediante una fórmula que sólo requiera una data mensual o anual de precipitación. A continuación, se muestran algunas de las fórmulas adaptadas para una precipitación media anual. 

<img src="https://user-images.githubusercontent.com/16768318/73694993-228d0d80-46d1-11ea-8bc4-9962963850b7.png">


Si bien es cierto, se usa ampliamente una precipitación media anual para estimar el **factor R** debido a la escasez de información, para este ejemplo se ha optado por  utilizar la fórmula desarrollada por **(Wischmeier & Smith, 1978)** debido a que se cuenta con una serie histórica de información de precipitación mensual. La fórmula es: 


<img src="https://user-images.githubusercontent.com/16768318/73695488-2b321380-46d2-11ea-8033-0063f27698d8.png" width = 50%>

In [0]:
# Monthly precipitation in mm at 1 km resolution: 
# https://zenodo.org/record/3256275#.XjibuDJKiM8
clim_rainmap = ee.Image("OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01")
year = clim_rainmap.reduce(ee.Reducer.sum())
R_monthly = ee.Image(10).pow(ee.Image(1.5).multiply(clim_rainmap.pow(2).divide(year).log10().subtract(-0.08188))).multiply(1.735)
factorR = R_monthly.reduce(ee.Reducer.sum())

In [0]:
center_coordinate = [0,0]
palette_rain = ["#450155", "#3B528C", "#21918D", "#5DCA63","#FFE925"]
mapdisplay(center_coordinate, {'Factor_R':factorR.getMapId({'min':0,'max':6000,'palette':palette_rain})},zoom_start=3)

### **2) Factor K**

A diferencia del factor R, el factor K muestra qué tan susceptible es el suelo
a la erosión hídrica, esto es determinado por las propiedades físicas y químicas del suelo, que dependen de las características de estos.Para determinar el factor K, existen una gran cantidad de fórmulas empíricas, adecuadas para diversos lugares del mundo y donde intervienen características del suelo como porcentaje de arena, limo, arcilla; estructura del suelo; contenido de carbono orgánico o materia orgánica; entre otros.

El factor K puede variar en una escala de 0 a 1, donde 0 indica suelos con la menor susceptibilidad a la erosión y 1 indica suelos altamente susceptibles a la erosión hídrica del suelo; cabe mencionar que esta escala fue hecha para el sistema de unidades americanas, y adaptándose al sistema internacional, la escala varía a normalmente entre 0 y 0.07.

A continuación, se muestran algunas ecuaciones para la estimación de este factor:

<img src="https://user-images.githubusercontent.com/16768318/73704444-039b7500-46eb-11ea-9ccd-b7850bb17911.png" width = 50%>
<img src="https://user-images.githubusercontent.com/16768318/73704442-039b7500-46eb-11ea-870c-a557ca50b777.png" width = 50%>
<img src="https://user-images.githubusercontent.com/16768318/73704443-039b7500-46eb-11ea-9469-104f04983dfd.png" width = 50%>


Para este ejemplo se ha optado por  utilizar la fórmula desarrollada por **Williams (1975)**.



In [0]:
# Cargamos toda la informacion necesaria para estimar el factor K
sand = ee.Image("OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02").select('b0')
silt = ee.Image('users/aschwantes/SLTPPT_I').divide(100)
clay = ee.Image("OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02").select('b0')
morg = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").select('b0').multiply(0.58)
sn1 = sand.expression('1 - b0 / 100', {'b0': sand})
orgcar = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").select('b0')

In [0]:
#Juntando todas las imagenes en una sola
soil = ee.Image([sand, silt, clay, morg, sn1, orgcar]).rename(['sand', 'silt', 'clay', 'morg', 'sn1', 'orgcar'] )

In [0]:
factorK = soil.expression(
  '(0.2 + 0.3 * exp(-0.0256 * SAND * (1 - (SILT / 100)))) * (1 - (0.25 * CLAY / (CLAY + exp(3.72 - 2.95 * CLAY)))) * (1 - (0.7 * SN1 / (SN1 + exp(-5.51 + 22.9 * SN1))))',
  {
    'SAND': soil.select('sand'),
    'SILT': soil.select('silt'),
    'CLAY': soil.select('clay'),
    'MORG': soil.select('morg'),
    'SN1':  soil.select('sn1'),
    'CORG': soil.select('orgcar')
  });

In [0]:
center_coordinate = [0,0]
palette_k = palette = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ]
viz_param_k = {'min': 0.0, 'max': 0.5, 'palette': palette_k};
mapdisplay(center_coordinate, {'Factor_K':factorK.getMapId(viz_param_k)},zoom_start=3)

### **3) Factor LS**

El factor LS expresa el efecto de la topografía local sobre la tasa de erosión del suelo, combinando los efectos de la longitud de la pendiente (L) y la inclinación de la pendiente (S). A medida que mayor sea la longitud de la pendiente, mayor será la cantidad de escorrentía acumulada y de la misma forma, mientras más pronunciada sea la pendiente de la superficie, mayor será la velocidad de la escorrentía, que influye directamente en la erosión.
Existen diversas metodologías basadas en SIG para calcular estos factores, como se pueden mostrar a continuación: 

<img src="https://user-images.githubusercontent.com/16768318/73706484-7ce99680-46f0-11ea-8e0e-5fbb4a00731d.png" width = 50%>

In [0]:
facc = ee.Image("WWF/HydroSHEDS/15ACC")
dem = ee.Image("WWF/HydroSHEDS/03CONDEM")
slope = ee.Terrain.slope(dem)

ls_factors = ee.Image([facc, slope]).rename(['facc','slope'])

In [0]:
factorLS = ls_factors.expression(
  '(FACC*270/22.13)**0.4*(SLOPE/0.0896)**1.3',
  {
    'FACC': ls_factors.select('facc'),
    'SLOPE': ls_factors.select('slope')     
  });

In [0]:
center_coordinate = [0,0]
palette_ls = palette = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ]
viz_param_k = {'min': 0, 'max': 100, 'palette': palette_ls};
mapdisplay(center_coordinate, {'Factor_LS':factorLS.getMapId(viz_param_k)},zoom_start=3)

### **4) Factor C**

El factor C se utiliza para determinar la eficacia relativa de los sistemas de manejo del suelo y de los cultivos en términos de prevención o reducción de la pérdida de suelo. Este factor indica cómo la cobertura vegetal y los cultivos afectarán la pérdida media anual de suelos y cómo se distribuirá el potencial de pérdida de suelos en el tiempo (Rahaman, 2015).

El valor de C depende del tipo de vegetación, la etapa de crecimiento y el porcentaje de cobertura. Valores más altos del factor C indican que no hay efecto de cobertura y pérdida de suelo, mientras que el menor valor de C significa un efecto de cobertura muy fuerte que no produce erosión.



In [0]:
ndvi_median = ee.ImageCollection("MODIS/006/MOD13A2").median().multiply(0.0001).select('NDVI')

In [0]:
geo_ndvi = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
]
l8_viz_params = {'palette':geo_ndvi,'min':0,'max': 0.8}
mapdisplay([0,0],{'composite_median':ndvi_median.getMapId(l8_viz_params)},zoom_start=3)

Otra forma de hallar este factor C, es haciendo una comparación entre el NDVI a partir de las fórmulas Van de Kniff (1999) [C1] y su adaptación para países asiáticos, que también se adecúan a la realidad de la costa peruana de Lin (2002) [C2]. Por último se tiene la ecuación formulada por De Jong(1994) [C3] adaptado a estudios de degradación de suelos en un entorno mediterráneo.
<center>
<img src="https://user-images.githubusercontent.com/16768318/73713048-e6bf6b80-4703-11ea-80b1-1940e6b55707.png" width = 50%>
</center>

In [0]:
factorC = ee.Image(0.805).multiply(ndvi_median).multiply(-1).add(0.431)

### **5) Calculo de la Erosion**

**A = R\*K\*LS\*C\*1**

  <img src="https://user-images.githubusercontent.com/16768318/73690808-1604b700-46c9-11ea-8bdd-43e0e490a0a3.gif">

In [0]:
erosion = factorC.multiply(factorR).multiply(factorLS).multiply(factorK)

In [0]:
geo_erosion = ["#00BFBF", "#00FF00", "#FFFF00", "#FF7F00", "#BF7F3F", "#141414"]
l8_viz_params = {'palette':geo_erosion,'min':0,'max': 6000}
mapdisplay([0,0],{'composite_median':erosion.getMapId(l8_viz_params)},zoom_start=3)

### **Descargar para cualquier parte del mundo**